### Paper figures

<b>Final code to make the figures for Zanowski et al., 2022:</b><br>
Zanowski, H., A. Jahn, S. Gu, Z. Liu, and T.M. Marchitto, (2022): Decomposition of deglacial Pacific radiocarbon age controls using an isotope-enabled ocean model, _Paleoceanography and Paleoclimatology_

1. Information about C-iTRACE and the model output can be found through the C-iTRACE website [here](https://sites.google.com/colorado.edu/citrace/home)<br>
2. Model output needed for the analyses can be also be found directly [here](https://doi.org/10.5065/hanq-bn92)<br>
3. C-iTRACE freshwater forcing time series can be found in ```citrace_fw.txt```, which is included in the github repository where you found this notebook
4. GLODAPv1 (Key et al., 2004) data is available [here](https://www.ncei.noaa.gov/access/ocean-carbon-data-system/oceans/glodap/GlopDV.html)
5. For supplementary figure S2, the model $\Delta^{14}$C output is regridded onto a 1˚x1˚ grid to be directly comparable to GLODAPv1. The regridded output is too large to put in the github repository, but the code to regrid the model output is provided in ```regrid_citrace_D14C.py```
6. For supplementary figures S3 and S4, the sediment core records are available as a compilation by Zhao et al., 2017 [here](https://www.ncei.noaa.gov/metadata/geoportal/rest/metadata/item/noaa-ocean-21390/html).

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import cmocean as cmo
import matplotlib.pyplot as plt
import carbon_isotope_functions as cif
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

Set up directories and dictionaries for the major paleo events of the deglaciation

In [ ]:
#Some saving parameters, etc
#data_direc='/glade/scratch/sgu28/ctrace_transient/ctrace_decadal_all/'
data_direc='/glade/work/zanowski/citrace/'
wrk_direc='/glade/work/zanowski/'
svdirec='/glade/u/home/zanowski/citrace/figures/paper_final/'

#Time periods dictionary, based on GICC05
#LGM: 23ka-18ka
#HS1: 18ka-14.7ka
#Bølling-Allerød: 14.7ka-12.9ka
#Younger Dryas: 12.9ka-11.7ka
#Holocene: 11.5ka onward
#Indices are for computations as well!
paleo_events={'LGM':{'yrs':[22,18],'idx':[0,400]},
              'HS1':{'yrs':[18,14.7],'idx':[400,730]},
              'BA':{'yrs':[14.7,12.9],'idx':[730,910]},
              'YD':{'yrs':[12.9,11.7],'idx':[910,1030]},
              'Holocene':{'yrs':[11.5,0],'idx':[1050,-1]}}

#Set up the indices for the four regions in original output space and the 1˚ regridded space
regions={'NWP':{'xid':[48,64],'yid':[71,81]},'NWP_r':{'xid':[135,190],'yid':[119,151]},
        'NEP':{'xid':[68,78],'yid':[71,81]},'NEP_r':{'xid':[210,240],'yid':[119,151]},
        'EEP':{'xid':[77,89],'yid':[35,63]},'EEP_r':{'xid':[240,280],'yid':[79,101]},
        'SWP':{'xid':[57,64],'yid':[17,26]},'SWP_r':{'xid':[170,190],'yid':[39,61]}}

Read in the model output for tracers, the freshwater forcing, and compute D14C ages, etc

In [ ]:
#Adapted region mask that cuts the Southern Ocean by basin
dsm=xr.open_dataset(wrk_direc+'citrace/CiTRACE_Adapted_Region_Mask.nc')
mask=dsm.region_mask
#Get the original model output
ds0=xr.open_dataset(data_direc+'ctrace.decadal.evo.subset.nc')
tlat=ds0.TLAT
tlon=ds0.TLONG
iage=ds0.IAGE
vage=ds0.VAGE
vagevmix=ds0.VAGEVMIX
zt=-0.01*ds0['z_t']
CO2atmo=ds0['ATM_CO2'].mean(dim=('nlat','nlon')) 
D14C=ds0['ABIO_D14Cocn']
D14Catmo=ds0['ABIO_D14Catm'].mean(dim=('nlat','nlon')) #This is fine to do the global mean 
#even though the atmo 14C values vary (somewhat) in the three lat bands across the globe.
#Correct for atmo D14C values by subtracting from ocean values
#DO AGE SUBTRACTION NOT D14C SUBTRACTION
D14C_ba=D14C-D14Catmo
#C14 age (for B-P)
D14C_age=cif.c14_age_godwin(D14C)
#C14 atmo age
D14Catmo_age=cif.c14_age_godwin(D14Catmo)
#C14 b-a age
D14C_ba_age=D14C_age-D14Catmo_age

####Model fw forcing time series
fw_ts=pd.read_table('citrace_fw.txt',sep=',')

<b>Figure 1:</b> GMOC (Global meridional overturning circulation) 50S, AMOC (Atlantic meridional overturning circulation) 25N, atmospheric D14C, Northern Hemisphere and Southern Hemisphere freshwater (fw) forcing time series

In [ ]:
#Compute AMOC and 'SMOC'--the min of the GMOC at 50S
amoc=ds0['MOC'][:,1,0]
#Get max AMOC at 25N--this is on the lat_aux_grid!
amoc_max=amoc[:,:,71].max('moc_z')
#MOC deep overturning cell min at 50S and upper overturning max--GLOBAL
gmoc=ds0['MOC'][:,1,0]
gmoc_min=gmoc[:,:,17].min('moc_z') #50.6S
#gmoc_max=gmoc[:,:,17].max('moc_z') #50.6S
#FW time series
nh_fw=fw_ts['NH_tot']
sh_fw=fw_ts['SH_tot']
fw_tme=fw_ts['Time']*1000

In [ ]:
tme=np.arange(22000,0,-10)
fs=9
tick_size=7
fig,ax=plt.subplots(3,1,sharex=True)
fig.set_size_inches(3.94,4.31)
axs=ax.flatten()
for a in axs:
    #Add time periods for Younger Dryas, Bølling-Allerød warm period, HS1
    for event in ['HS1','BA','YD']:
        a.axvspan(xmin=paleo_events[event]['yrs'][0]*1000,xmax=paleo_events[event]['yrs'][1]*1000,
                      facecolor='0.7',edgecolor='darkmagenta',alpha=0.5)
l0=axs[0].plot(tme,D14Catmo,color='k',lw='1.5',label='$\Delta^{14}$C')
a0_1=axs[0].twinx()
l01=a0_1.plot(tme,CO2atmo,color='darkgreen',lw='1',label='CO$_{2}$')
a0_1.tick_params(labelsize=tick_size)
t0=axs[0].set_title('Prescribed Atmospheric $\Delta^{14}$C and CO$_{2}$',fontsize=fs)
t0.set_y(0.96)
axs[0].set_ylim(-30,510)
axs[0].set_ylabel('$\Delta^{14}$C (‰)',fontsize=fs)
a0_1.set_ylabel('CO$_{2}$ (ppm)',rotation=270,va='bottom',fontsize=fs)
a0_1.set_ylim(180,320)
lns0=l0+l01
lbs0=[l.get_label() for l in lns0]
axs[0].legend(lns0,lbs0,loc='upper right',frameon=False,ncol=2,handlelength=1,
              handletextpad=0.3,borderaxespad=0.01,columnspacing=0.8,fontsize=tick_size)


l1=axs[1].plot(fw_tme,nh_fw,label='FW$_{tot}$ NH',color='goldenrod',lw=1)
a1_1=axs[1].twinx()
l2=a1_1.plot(tme,amoc_max,label='AMOC',color='navy',lw=0.75)
a1_1.tick_params(labelsize=tick_size)
a1_1.set_ylabel('AMOC (Sv)',rotation=270,va='bottom',fontsize=fs)
axs[1].set_ylim(0,22)
a1_1.set_ylim(0,23.5)
t1=axs[1].set_title('AMOC at 25$\degree$N and NH FW Forcing',fontsize=fs)
t1.set_y(0.97)
lns1=l1+l2
lbs1=[l.get_label() for l in lns1]
axs[1].legend(lns1,lbs1,loc='upper right',frameon=False,handlelength=1,
              handletextpad=0.3,borderaxespad=0.01,columnspacing=0.8,
              labelspacing=0.1,fontsize=tick_size,ncol=2)

l3=axs[-1].plot(fw_tme,sh_fw,label='FW$_{tot}$ SH',color='goldenrod',lw=1)
a2_1=axs[-1].twinx()
l4=a2_1.plot(tme,gmoc_min,label='MOC',color='navy',lw=0.75)
axs[-1].set_ylabel('FW forcing (m kyr$^{-1}$)',fontsize=fs)
a2_1.set_ylabel('MOC (Sv)',rotation=270,va='bottom',fontsize=fs)
a2_1.tick_params(labelsize=tick_size)
axs[-1].yaxis.set_label_coords(-0.1,1.2)
axs[-1].set_ylim(0,65)
a2_1.set_ylim(-6,0)
t2=axs[-1].set_title('Deep MOC at 50$\degree$S and SH FW Forcing',fontsize=fs)
t2.set_y(0.97)
lns2=l3+l4
lbs2=[l.get_label() for l in lns2]
axs[-1].legend(lns2,lbs2,loc='upper right',frameon=False,handlelength=1,
               handletextpad=0.3,borderaxespad=0.01,columnspacing=0.8,
               labelspacing=0.1,fontsize=tick_size,ncol=2)

axs[-1].set_xticks(np.arange(22000,-1,-2000))
axs[-1].set_xticklabels(np.arange(22,-1,-2))
axs[-1].set_xlim(20000,0)
axs[-1].set_xlabel(r'Time (ka BP)',fontsize=fs)

plt.subplots_adjust(hspace=0.3,right=0.86,left=0.14)
ann=[axs[0].annotate(event,(paleo_events[event]['yrs'][0]*1000-0.5*1000*(paleo_events[event]['yrs'][0]-paleo_events[event]['yrs'][1]),440),
                        xycoords='data',fontsize=tick_size,ha='center',va='bottom',color='k',zorder=6) for event in ['HS1','BA','YD']]
#letters
letts=['a)','b)','c)']
for i in range(len(axs)):
    axs[i].annotate(letts[i],(-0.02,1.07),xycoords='axes fraction',fontsize=fs)
    axs[i].tick_params(labelsize=tick_size)
plt.savefig(svdirec+'figure1.pdf')

<b>Figure 2:</b> Model zonal mean Pacific D14C at 3 times (20ka, 10ka, PD)

In [ ]:
plat=ds0['TLAT'][:,60] #mid pacific
zt=-0.01*ds0['z_t']
#22ka (start)
D14C_20=D14C[200].where(mask==2).mean('nlon')
#11ka
D14C_10=D14C[1200].where(mask==2).mean('nlon')
#present (0ka)
D14C_0=D14C[-1].where(mask==2).mean('nlon')
dat=[D14C_20,D14C_10,D14C_0]

In [ ]:
fs=10
tick_size=8
fig,ax=plt.subplots(1,3,sharey=True)
fig.set_size_inches(7,2)
axs=ax.flatten()
lbs=['20 ka BP','10 ka BP','PD (0 ka BP)']
for i in range(0,len(axs)):
    plt.sca(axs[i])
    axs[i].set_facecolor('0.7')
    cs=plt.contourf(plat,zt,dat[i],levels=np.arange(-300,301,25),extend='both',cmap=plt.cm.BrBG_r) #Spectral_r
    axs[i].set_xlim(-80,65)
    axs[i].set_ylim(-5500,0)
    axs[i].set_xticklabels(['','50$\degree$S','0$\degree$','50$\degree$N'])
    axs[i].tick_params(labelsize=tick_size)
    t=plt.title(lbs[i],fontsize=fs)
    t.set_y(0.985)
rght=0.86
plt.subplots_adjust(left=0.11,right=rght,wspace=0.1,hspace=0.1,top=0.9)
#axs[1].set_xlabel('Latitude')
axs[0].set_ylabel('Depth (m)',fontsize=fs)
#ts=plt.suptitle(r'Zonal Mean Abiotic $\Delta^{14}$C')
#ts.set_y(0.98)
#ts.set_x(0.492)

#Colorbar
cax = fig.add_axes([rght+0.015, axs[-1].get_position().y0, 0.02, axs[0].get_position().y1-axs[-1].get_position().y0]) #x,y,w,h
cbar=plt.colorbar(cs,cax=cax,orientation='vertical')
cbar.set_label(r'$\Delta^{14}$C (‰)',rotation=270,va='bottom',fontsize=fs) #shift-option-R for permil
cbar.ax.tick_params(labelsize=tick_size)
letts=['a)','b)','c)']
for i in range(len(axs)):
    axs[i].annotate(letts[i],(0.0,1.047),xycoords='axes fraction',fontsize=fs)
plt.savefig(svdirec+'figure2.pdf')

<b>Figure 3:</b> Map of averaging regions and transport transects

In [ ]:
tick_size=9
fs=10
fig=plt.figure()
fig.set_size_inches(2.75,2.75)
#fig.set_size_inches(3,3)
#map axes
ax=fig.add_subplot(111,projection=ccrs.PlateCarree(central_longitude=200))
ax.set_extent([100,310,-90, 90], ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, facecolor='0.8',edgecolor='None',linewidth=0.5)

t0=ax.set_title('Averaging and Transport Regions',fontsize=fs)
t0.set_y(1.0)
#averaging boxes
b1=ax.plot([135,190,190,135,135],[30,30,63,63,30],color='k',linewidth=1,transform=ccrs.PlateCarree()) #lower left start, clockwise
b2=ax.plot([209,241,241,209,209],[30,30,63,63,30],color='k',linewidth=1,transform=ccrs.PlateCarree()) #lower left start, clockwise
b3=ax.plot([240,281,281,240,240],[-10,-10,12,12,-10],color='k',linewidth=1,transform=ccrs.PlateCarree()) #lower left start, clockwise
b4=ax.plot([170,191,191,170,170],[-50,-50,-30,-30,-50],color='k',linewidth=1,transform=ccrs.PlateCarree()) #lower left start, clockwise
gl=ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='gray', alpha=0.5, linestyle='-')

#Transect lines
#30n_wbc=30N, 107E-143E ytrans_full[:,:,71,40:51].sum('nlon')
ax.plot([120,143],[30,30],transform=ccrs.PlateCarree(),lw=3,alpha=0.5,color='darkmagenta',zorder=1)
#30n_int=30N, 143E-248E ytrans_full[:,:,71,52:80].sum('nlon')
ax.plot([143,245],[30,30],transform=ccrs.PlateCarree(),lw=3,alpha=0.5,color='darkgoldenrod',zorder=1)
#30s_wbc=30S, 143E-194E ytrans_full[:,:,24,50:65].sum('nlon')
ax.plot([151,194],[-30,-30],transform=ccrs.PlateCarree(),lw=3,alpha=0.5,color='darkmagenta',zorder=1)
#30s_int=30S, 194E-302E ytrans_full[:,:,24,65:95].sum('nlon')
ax.plot([194,289],[-30,-30],transform=ccrs.PlateCarree(),lw=3,alpha=0.5,color='darkgoldenrod',zorder=1)

#Map labels, etc
gl.xlabels_top=False #don't label the top or right with lat/lon
gl.ylabels_right=False
gl.xlines=False #turn off the grid lines in x
gl.ylines=False #turn off the grid lines in y
gl.xlocator=mticker.FixedLocator([120,180,-120,-60]) #these need not be evenly spaced
gl.ylocator=mticker.FixedLocator([-90,-60,-30,0,30,60,90])
gl.xformatter=LONGITUDE_FORMATTER
gl.yformatter=LATITUDE_FORMATTER
gl.xlabel_style = {'size': tick_size, 'color': 'k'}
gl.ylabel_style = {'size': tick_size, 'color': 'k'}

plt.subplots_adjust(left=0.15,right=0.95)

#Annotate boxes
tts=['NW Pacific', 'NE Pacific', 'East Pacific', 'SW Pacific']
locs=[(0.12,0.88),(0.52,0.88),(0.62,0.59),(0.26,0.13)]
#locs=[(0.12,0.88),(0.50,0.88),(0.65,0.6),(0.26,0.14)] v2 for 3x3
for i in range(0,len(tts)):
    ax.annotate(tts[i],locs[i],xycoords='axes fraction',fontsize=tick_size)

plt.savefig(svdirec+'figure3.pdf')

<b>Figure 4:</b> Model Pacific zonal mean age vs ventilation age and ideal age at 3 times (20ka, 10ka, PD)

In [ ]:
vage=ds0.VAGE
iage=ds0.IAGE
plat=ds0['TLAT'][:,60] #mid pacific; 
zt=-0.01*ds0['z_t']
#20ka (start)
D14C_age_20=D14C_ba_age[200].where(mask==2).mean('nlon')
iage_20=iage[200].where(mask==2).mean('nlon')
vage_20=vage[200].where(mask==2).mean('nlon')
#10ka
D14C_age_10=D14C_ba_age[1200].where(mask==2).mean('nlon')
iage_10=iage[1200].where(mask==2).mean('nlon')
vage_10=vage[1200].where(mask==2).mean('nlon')
#present (0ka)
D14C_age_0=D14C_ba_age[-1].where(mask==2).mean('nlon')
iage_0=iage[-1].where(mask==2).mean('nlon')
vage_0=vage[-1].where(mask==2).mean('nlon')

dat=[D14C_age_20,D14C_age_10,D14C_age_0]
datv=[vage_20,vage_10,vage_0]
dati=[iage_20,iage_10,iage_0]

In [ ]:
fig,ax=plt.subplots(3,3,sharey=True,sharex=True)
fig.set_size_inches(6,4.0)
axs=ax.flatten()
tts=['20 ka BP','10 ka BP ','PD (0 ka BP)']
fs=9
tick_size=8
rght=0.85
plt.subplots_adjust(left=0.2,right=rght,wspace=0.1,hspace=0.2,top=0.9)
cmap=plt.cm.PuBuGn
levs=np.arange(0,3501,250)
for i in range(0,3):
    plt.sca(axs[i])
    axs[i].set_facecolor('0.7')
    cs=plt.contourf(plat,zt,dat[i],levels=levs,extend='max',cmap=cmap) #Spectral_r
    ts=plt.title(tts[i],fontsize=fs)
    axs[i].tick_params(labelsize=tick_size)
    ts.set_y(0.96)
    plt.sca(axs[i+3])
    axs[i+3].set_facecolor('0.7')
    cs1=plt.contourf(plat,zt,dati[i],levels=levs,extend='max',cmap=cmap) #Spectral_r
    axs[i+3].tick_params(labelsize=tick_size)
    plt.sca(axs[i+6])
    axs[i+6].set_facecolor('0.7')
    cs2=plt.contourf(plat,zt,datv[i],levels=levs,extend='max',cmap=cmap) #Spectral_r
    axs[i+6].tick_params(labelsize=tick_size)
    
#Colorbar
cax = fig.add_axes([rght+0.015, axs[-1].get_position().y0, 0.018, axs[2].get_position().y1-axs[-1].get_position().y0]) #x,y,w,h
cbar=plt.colorbar(cs,cax=cax,orientation='vertical')
cbar.set_label('Age (yr)',fontsize=tick_size,rotation=270,va='bottom') #shift-option-R for permil
cbar.ax.tick_params(labelsize=tick_size)

axs[-1].set_xlim(-80,65)
axs[-1].set_ylim(-5500,0)
axs[-1].set_yticks(np.arange(-5000,1,1000))
axs[-1].set_xticklabels(['','50$\degree$S','0$\degree$','50$\degree$N'],fontsize=tick_size)
#axs[7].set_xlabel('Latitude')
axs[3].set_ylabel('Depth (m)',fontsize=tick_size)
#axs[6].yaxis.set_label_coords(-0.35,1.1)

#Tracer labels
lbs=['$\mathbf{\Delta^{14}}$C', 'Ideal', 'Ventilation']
for i,a in enumerate(axs[[0,3,6]]):
    a.annotate(lbs[i],(-0.75,0.5),xycoords='axes fraction',fontsize=fs,fontweight='bold',rotation=90,va='center')

#Letters
letts=['a)','b)','c)','d)','e)','f)','g)','h)','i)']
for i in range(0,len(axs)):
    axs[i].annotate(letts[i],(0,1.05),xycoords='axes fraction',fontsize=fs)

plt.savefig(svdirec+'figure4.pdf')

<b> Figure 5:</b> meridional transports across 30N and 30S split into interior and WBC components using a single longitude

In [ ]:
v=0.01*ds0.VVEL.where(mask==2) #pac only
rho=1000*ds0.PD.where(mask==2) #kg/m3
dz=0.01*ds0.dz
dy=0.01*ds0.DYU
#Transport
ytrans_full=v*dz*dy
ytrans=ytrans_full.sum('nlon')
ytrans_s=ytrans[:,:,24]
ytrans_n=ytrans[:,:,71]
ytrans_s_wbc=ytrans_full[:,:,24,50:65].sum('nlon')
ytrans_s_int=ytrans_full[:,:,24,65:95].sum('nlon')
ytrans_n_wbc=ytrans_full[:,:,71,40:51].sum('nlon')
ytrans_n_int=ytrans_full[:,:,71,52:80].sum('nlon')

In [ ]:
dat=[1e-06*ytrans_n_wbc.T,1e-06*ytrans_n_int.T,1e-06*ytrans_s_wbc.T,1e-06*ytrans_s_int.T]
tme=np.arange(22000,0,-10)
zt=-0.01*ds0['z_t']
cmp=plt.cm.RdBu_r
clevs2=np.arange(-1.05,1.06,0.1)
tts=['30$\degree$N DWBC', '30$\degree$N Interior', '30$\degree$S DWBC', '30$\degree$S Interior']
tick_size=8 #ticklabel fontsize
fs=9 #title/text fontsize

fig=plt.figure(constrained_layout=False)
gs1=fig.add_gridspec(nrows=2,ncols=4,wspace=0.12)
fig.set_size_inches(8,2.3)
axs2=[fig.add_subplot(gs1[:,i]) for i in range(0,4)]

for i in range(0,4):
    cs2=axs2[i].contourf(tme,zt[45:],dat[i][45:],levels=clevs2,extend='both',cmap=cmp)
    axs2[i].set_ylim(-5000,-2000)
    axs2[i].set_xticks(np.arange(22000,-1,-2000))
    axs2[i].set_xticklabels(np.arange(22,-1,-2))
    axs2[i].set_xlim(20000,0)
    axs2[i].set_ylim(-5375,-2000)
    axs2[i].set_yticks(np.arange(-5000,-1999,1000))
    axs2[i].tick_params(labelsize=tick_size)
    t1=axs2[i].set_title(tts[i],fontsize=fs)
    t1.set_y(0.97)
    for lb in axs2[i].get_xticklabels()[::2]:
        lb.set_visible(False)
    if i>0:
        axs2[i].set_yticklabels('')

rght=0.9
plt.subplots_adjust(top=0.9,right=rght,left=0.08,bottom=0.2)
axs2[1].set_xlabel('Time (ka BP)',fontsize=tick_size)
axs2[1].xaxis.set_label_coords(1.06,-0.18)
axs2[0].set_ylabel('Depth (m)',fontsize=tick_size)
#axs2[0].yaxis.set_label_coords(-0.44,0.9)
#ts=plt.suptitle(r'Volume transport across 30$\degree$N and 30$\degree$S')
#ts.set_x(0.48)
#ts.set_y(0.99)

#colorbar
cax2=fig.add_axes([rght+0.01, axs2[-1].get_position().y0, 0.012, axs2[0].get_position().y1-axs2[-1].get_position().y0]) #x,y,w,h
cbar2=plt.colorbar(cs2,cax=cax2,orientation='vertical')
cbar2.set_label('Volume Transport (Sv)',fontsize=tick_size,rotation=270,va='bottom')
cbar2.ax.tick_params(labelsize=tick_size)

letts=['a)','b)','c)','d)']
for i in range(0,len(axs2)):
    axs2[i].annotate(letts[i],(0,1.026),xycoords='axes fraction',fontsize=fs)
plt.savefig(svdirec+'figure5.pdf')

<b>Figure 6:</b> Profiles of D14C age (benthic - atmospheric; B-A), and benthic (model bottom) ventilation and ideal age in the four regions at 20 ka BP, 10 ka BP, and PD

In [ ]:
#Compute region means (used also for Figures 7,9,10)
#Get the regions in the model
nwp=D14C_ba_age[:,:,regions['NWP']['yid'][0]:regions['NWP']['yid'][1],regions['NWP']['xid'][0]:regions['NWP']['xid'][1]]
nep=D14C_ba_age[:,:,regions['NEP']['yid'][0]:regions['NEP']['yid'][1],regions['NEP']['xid'][0]:regions['NEP']['xid'][1]]
eqpac=D14C_ba_age[:,:,regions['EEP']['yid'][0]:regions['EEP']['yid'][1],regions['EEP']['xid'][0]:regions['EEP']['xid'][1]]
swp=D14C_ba_age[:,:,regions['SWP']['yid'][0]:regions['SWP']['yid'][1],regions['SWP']['xid'][0]:regions['SWP']['xid'][1]]
#Get the regions in the model
nwpi=iage[:,:,regions['NWP']['yid'][0]:regions['NWP']['yid'][1],regions['NWP']['xid'][0]:regions['NWP']['xid'][1]]
nepi=iage[:,:,regions['NEP']['yid'][0]:regions['NEP']['yid'][1],regions['NEP']['xid'][0]:regions['NEP']['xid'][1]]
eqpaci=iage[:,:,regions['EEP']['yid'][0]:regions['EEP']['yid'][1],regions['EEP']['xid'][0]:regions['EEP']['xid'][1]]
swpi=iage[:,:,regions['SWP']['yid'][0]:regions['SWP']['yid'][1],regions['SWP']['xid'][0]:regions['SWP']['xid'][1]]
#Get the regions in the model
nwpv=vage[:,:,regions['NWP']['yid'][0]:regions['NWP']['yid'][1],regions['NWP']['xid'][0]:regions['NWP']['xid'][1]]
nepv=vage[:,:,regions['NEP']['yid'][0]:regions['NEP']['yid'][1],regions['NEP']['xid'][0]:regions['NEP']['xid'][1]]
eqpacv=vage[:,:,regions['EEP']['yid'][0]:regions['EEP']['yid'][1],regions['EEP']['xid'][0]:regions['EEP']['xid'][1]]
swpv=vage[:,:,regions['SWP']['yid'][0]:regions['SWP']['yid'][1],regions['SWP']['xid'][0]:regions['SWP']['xid'][1]]

#Calculate the region mean D14C age
nwpm=nwp.mean(dim=('nlat','nlon'))
nepm=nep.mean(dim=('nlat','nlon'))
eqpacm=eqpac.mean(dim=('nlat','nlon'))
swpm=swp.mean(dim=('nlat','nlon'))

#Calculate the region mean ideal age
nwpmi=nwpi.mean(dim=('nlat','nlon'))
nepmi=nepi.mean(dim=('nlat','nlon'))
eqpacmi=eqpaci.mean(dim=('nlat','nlon'))
swpmi=swpi.mean(dim=('nlat','nlon'))

#Calculate the region mean ventilation age
nwpmv=nwpv.mean(dim=('nlat','nlon'))
nepmv=nepv.mean(dim=('nlat','nlon'))
eqpacmv=eqpacv.mean(dim=('nlat','nlon'))
swpmv=swpv.mean(dim=('nlat','nlon'))

In [ ]:
#Bottom of swp and eqpac in the model are at index levels 54 and 58
time_ids=[200,1200,-1]
c_age=[[nwpm[i,:] for i in time_ids],[nepm[i,:] for i in time_ids],
       [eqpacm[i,:] for i in time_ids],[swpm[i,:] for i in time_ids]]
i_age=[[nwpmi[i,:] for i in time_ids],[nepmi[i,:] for i in time_ids],
       [eqpacmi[i,:] for i in time_ids],[swpmi[i,:] for i in time_ids]]
v_age=[[nwpmv[i,:] for i in time_ids],[nepmv[i,:] for i in time_ids],
       [eqpacmv[i,:] for i in time_ids],[swpmv[i,:] for i in time_ids]]

tts=['NW Pacific', 'NE Pacific', 'East Pacific', 'SW Pacific']
yrs=['20 ka', '10 ka' ,'0 ka']
ylms=[[-5375,0],[-5375,0],[-4125,0],[-5125,0]]
yts=[np.arange(-5000,1,1000),np.arange(-5000,1,1000),
     np.arange(-4000,1,1000),np.arange(-5000,1,1000)]
fs=9
tick_size=6

fig=plt.figure(constrained_layout=False)
fig.set_size_inches(4,3)
outer_grid=fig.add_gridspec(2,2,wspace=0.35, hspace=0.35,left=0.16,right=0.96)

axs=[]
for a in range(4):
    # gridspec inside gridspec
    inner_grid = outer_grid[a].subgridspec(2,3, wspace=0.2, hspace=0)
    axs.append([fig.add_subplot(inner_grid[:,i]) for i in range(0,3)])  #subplots for the inner grid.
                                           
for j in range(0,4):
    for i in range(0,3):
        plt.sca(axs[j][i])
        l1=plt.plot(0.001*i_age[j][i],zt,color='goldenrod',lw=1,label='Ideal age')
        l2=plt.plot(0.001*v_age[j][i],zt,color='g',lw=1,label='Ventilation age')
        l3=plt.plot(0.001*c_age[j][i],zt,color='navy',lw=1,label='$\Delta^{14}$C age')        
        axs[j][i].set_ylim(ylms[j])
        axs[j][i].set_xlim(0,5)
        axs[j][i].set_xticks([0,2.5,5])
        axs[j][i].set_xticklabels([0,'',5])
        axs[j][i].set_yticks(yts[j])
        axs[j][i].tick_params(labelsize=tick_size)      
        t1=axs[j][i].set_title(yrs[i],fontsize=tick_size+1)
        t1.set_y(0.93)
        axs[j][i].set_ylim(ylms[j])
        if i>0:
            axs[j][i].set_yticklabels('')
        if j<2:
            axs[j][i].set_xticklabels('')
axs[2][-1].set_xlabel('Age (kyr)',fontsize=fs)
axs[2][-1].xaxis.set_label_coords(1.6,-0.25)
axs[2][0].set_ylabel('Depth (m)',fontsize=fs)
axs[2][0].yaxis.set_label_coords(-1.2,1.2)
        
letts=['a)','b)','c)','d)']
for j in range(0,len(axs)):
    axs[j][0].annotate(letts[j],(-0.3,1.13),xycoords='axes fraction',fontsize=fs)
    ts=axs[j][1].annotate(tts[j],(-0.35,1.13),xycoords='axes fraction',fontsize=fs)

plt.savefig(svdirec+'figure6.pdf')

<b>Figure 7:</b> Four regions: D14C (B-A), and benthic ventilation age and ideal age time series

In [ ]:
#Bottom of swp and eqpac are at index levels 54 and 58
c_age=[nwpm[:,-1],nepm[:,-1],eqpacm[:,54],swpm[:,58]]
i_age=[nwpmi[:,-1],nepmi[:,-1],eqpacmi[:,54],swpmi[:,58]]
v_age=[nwpmv[:,-1],nepmv[:,-1],eqpacmv[:,54],swpmv[:,58]]

tme=np.arange(22000,0,-10)
tts=['NW Pacific (5375 m)', 'NE Pacific (5375 m)', 
     'East Pacific (4125 m)', 'SW Pacific (5125 m)']
fs=10
tick_size=8

fig,ax=plt.subplots(2,2,sharex=True,sharey=True)
fig.set_size_inches(4.875,3)
axs=ax.flatten()
for i in range(0,4):
    plt.sca(axs[i])
    for event in ['HS1','BA','YD']:
        axs[i].axvspan(xmin=paleo_events[event]['yrs'][0]*1000,xmax=paleo_events[event]['yrs'][1]*1000,
                      facecolor='0.7',edgecolor='k',alpha=0.5)
    plt.plot(tme,i_age[i],color='goldenrod',lw=1,label='Ideal age')
    plt.plot(tme,v_age[i],color='g',lw=1,label='Ventilation age')
    plt.plot(tme,c_age[i],color='navy',lw=1,label='$\Delta^{14}$C age')
    ts=axs[i].set_title(tts[i],fontsize=fs)
    ts.set_y(0.95)
    axs[i].tick_params(labelsize=tick_size)

axs[0].set_ylim([0,5100])

axs[-1].set_xticks(np.arange(22000,-1,-2000))
axs[-1].set_xticklabels(np.arange(22,-1,-2))
axs[-1].set_xlim(20000,0)
axs[2].set_xlabel('Time (ka BP)',fontsize=fs)
axs[2].set_ylabel('Age (yr)',fontsize=fs)

plt.subplots_adjust(left=0.13, wspace=0.11,hspace=0.3,right=0.98)
axs[2].xaxis.set_label_coords(1.05,-0.25)
axs[2].yaxis.set_label_coords(-0.25,1.15)

axs[-1].legend(loc='upper right',frameon=False,handlelength=0.9,
              handletextpad=0.3,borderaxespad=0.01,labelspacing=0.1,
              fontsize=tick_size-1)

ann=[axs[-1].annotate(event,(paleo_events[event]['yrs'][0]*1000-0.5*1000*(paleo_events[event]['yrs'][0]-paleo_events[event]['yrs'][1]),4600),
                        xycoords='data',fontsize=5,ha='center',va='bottom',color='k',zorder=6) for event in ['HS1','BA','YD']]
letts=['a)','b)','c)','d)']
for i in range(0,len(axs)):
    axs[i].annotate(letts[i],(0,1.04),xycoords='axes fraction',fontsize=fs)

plt.savefig(svdirec+'figure7.pdf')

<b>Figure 8:</b> Depth vs time Hovmöller of oxygen in the four regions

In [ ]:
#Get the regions in the model
nwpo=ds0.O2[:,:,regions['NWP']['yid'][0]:regions['NWP']['yid'][1],regions['NWP']['xid'][0]:regions['NWP']['xid'][1]].mean(dim=('nlat','nlon'))
nepo=ds0.O2[:,:,regions['NEP']['yid'][0]:regions['NEP']['yid'][1],regions['NEP']['xid'][0]:regions['NEP']['xid'][1]].mean(dim=('nlat','nlon'))
eqpaco=ds0.O2[:,:,regions['EEP']['yid'][0]:regions['EEP']['yid'][1],regions['EEP']['xid'][0]:regions['EEP']['xid'][1]].mean(dim=('nlat','nlon'))
swpo=ds0.O2[:,:,regions['SWP']['yid'][0]:regions['SWP']['yid'][1],regions['SWP']['xid'][0]:regions['SWP']['xid'][1]].mean(dim=('nlat','nlon'))

In [ ]:
dat=[nwpo.T,nepo.T,eqpaco.T,swpo.T]
tme=np.arange(22000,0,-10)
zt=-0.01*ds0['z_t']
cmp=plt.cm.PuBuGn
clevs=np.arange(0,241,15)
yts=[np.arange(-5000,1,1000),np.arange(-5000,1,1000),np.arange(-4000,1,1000),np.arange(-5000,1,1000)]
ylms=[[-5375,0],[-5375,0],[-4125,0],[-5125,0]]
tts=['NW Pacific','NE Pacific','East Pacific','SW Pacific']
fs=10
tick_size=8

fig,ax=plt.subplots(2,2,sharex=True)
fig.set_size_inches(6,3.75)
axs=ax.flatten()
for i in range(0,4):
    cs=axs[i].contourf(tme,zt,dat[i],levels=clevs,cmap=cmp,extend='max') 
    cs2=axs[i].contour(tme,zt,dat[i],levels=[70],linewidths=1.5,colors='darkmagenta')
    #cs3=axs[i].contour(tme,zt,dat[i],levels=[5,10],linewidths=1,colors='purple',linestyles=':')
    axs[i].set_ylim(ylms[i])
    axs[i].set_yticks(yts[i])
    ts=axs[i].set_title(tts[i],fontsize=fs)
    ts.set_y(0.96)

axs[0].set_xlim(20000,0)
axs[0].set_xticks(np.arange(20000,-1,-2000))
axs[0].set_xticklabels(np.arange(20,-1,-2))

rght=0.85
plt.subplots_adjust(wspace=0.3,top=0.9,right=rght,hspace=0.2)
axs[2].set_xlabel('Time (ka BP)',fontsize=fs)
axs[2].xaxis.set_label_coords(1.15,-0.2)
axs[2].set_ylabel('Depth (m)',fontsize=fs)
axs[2].yaxis.set_label_coords(-0.3,1.1)
#ts=plt.suptitle(r'Ratio of Ideal Age to $\Delta^{14}$C Age')
#ts.set_x(0.48)
#ts.set_y(0.99)
#colorbar()
cax=fig.add_axes([rght+0.02, axs[-1].get_position().y0, 0.03, axs[0].get_position().y1-axs[-1].get_position().y0]) #x,y,w,h
cbar=plt.colorbar(cs,cax=cax,orientation='vertical')
cbar.set_label('O$_{2}$ (mmol m$^{-3}$)',fontsize=fs,rotation=270,va='bottom')
cbar.ax.tick_params(labelsize=tick_size)

letts=['a)','b)','c)','d)']
for i in range(0,len(axs)):
    axs[i].annotate(letts[i],(0,1.03),xycoords='axes fraction',fontsize=fs)
    axs[i].tick_params(labelsize=tick_size)

plt.savefig(svdirec+'figure8.pdf')

<b>Figure 9:</b> B-A D14C age at the surface and bottom in the 4 regions

In [ ]:
# With atmo adjustment--no surface subtraction!!
#Bottom of swp and eqpac are at index levels 54 and 58
c_ageb=[nwpm[:,-1],nepm[:,-1],eqpacm[:,54],swpm[:,58]]
c_aget=[nwpm[:,0],nepm[:,0],eqpacm[:,0],swpm[:,0]]
c_age=[c_aget,c_ageb]

lbs=['Surface','Bottom']
tme=np.arange(22000,0,-10)
tts=['NW Pacific', 'NE Pacific', 'East Pacific', 'SW Pacific']
fs=9
tick_size=6

fig=plt.figure(constrained_layout=False)
fig.set_size_inches(5,2.5)
outer_grid=fig.add_gridspec(2,2,wspace=0.2,hspace=0.4,left=0.12,right=0.96,bottom=0.12)

axs=[]
for a in range(4):
    # gridspec inside gridspec
    inner_grid = outer_grid[a].subgridspec(2,1, hspace=0.3)
    axs.append([fig.add_subplot(inner_grid[i,:]) for i in range(0,2)])  #subplots for the inner grid.
    
for i in range(0,2):
    for j in range(0,4):
        plt.sca(axs[j][i])
        for event in ['HS1','BA','YD']:
            axs[j][i].axvspan(xmin=paleo_events[event]['yrs'][0]*1000,xmax=paleo_events[event]['yrs'][1]*1000,
                      facecolor='0.7',edgecolor='k',alpha=0.5)
        l1=plt.plot(tme,c_age[i][j],color='navy',lw=1)  
        axs[j][i].set_xticks(np.arange(22000,-1,-2000))
        axs[j][i].set_xticklabels(np.arange(22,-1,-2))
        axs[j][i].set_xlim(20000,0)
        axs[j][i].tick_params(labelsize=tick_size)      
        t1=axs[j][0].set_title(tts[j],fontsize=fs)
        t1.set_y(0.9)
        plt.sca(axs[j][1])  
        if j<2:
            axs[j][i].set_xticklabels('')
        if i<1:
            axs[j][i].set_xticklabels('')
axs[2][-1].set_xlabel('Time (ka BP)',fontsize=fs-1)
axs[2][-1].xaxis.set_label_coords(1.1,-0.55)
axs[2][0].set_ylabel('B-A $\Delta^{14}$C age',fontsize=fs-1)
axs[2][0].yaxis.set_label_coords(-0.2,1.33)
        
letts=['a)','b)','c)','d)']
for j in range(0,len(axs)):
    axs[j][0].annotate(letts[j],(0,1.15),xycoords='axes fraction',fontsize=fs)

ann=[axs[1][0].annotate(event,(paleo_events[event]['yrs'][0]*1000-0.5*1000*(paleo_events[event]['yrs'][0]-paleo_events[event]['yrs'][1]),570),
                        xycoords='data',fontsize=5,ha='center',va='bottom',color='k',zorder=6) for event in ['HS1','BA','YD']]    

for j in range(0,len(axs)):
    axs[j][0].annotate('Surface',(0.82,0.7),xycoords='axes fraction',fontsize=tick_size)
    axs[j][1].annotate('Bottom',(0.82,0.7),xycoords='axes fraction',fontsize=tick_size)
    axs[j][0].set_ylim(0,750)
    axs[j][1].set_ylim(1000,5000)

plt.savefig(svdirec+'figure9.pdf')

<b>Figure 10:</b> Surface reservoir age effect in four regions: depth vs time Hovmöller of the ratio of ideal age to D14C age (%)

In [ ]:
nwpr=(nwpmi/nwpm).T
nepr=(nepmi/nepm).T
eqpacr=(eqpacmi/eqpacm).T
swpr=(swpmi/swpm).T

In [ ]:
dat=[nwpr,nepr,eqpacr,swpr]
tme=np.arange(22000,0,-10)
zt=-0.01*ds0['z_t']
#cmp=plt.cm.plasma_r
#cmp=plt.cm.PuBuGn
#cmp=cmo.cm.curl
cmp=plt.cm.PRGn_r
#clevs=np.arange(0,1.1,0.1)
clevs=np.arange(0.05,0.96,0.05)
yts=[np.arange(-5000,1,1000),np.arange(-5000,1,1000),np.arange(-4000,1,1000),np.arange(-5000,1,1000)]
ylms=[[-5375,0],[-5375,0],[-4125,0],[-5125,0]]
tts=['NW Pacific','NE Pacific','East Pacific','SW Pacific']
fs=10
tick_size=8

fig,ax=plt.subplots(2,2,sharex=True)
fig.set_size_inches(6,3.75)
axs=ax.flatten()
for i in range(0,4):
    cs=axs[i].contourf(tme,zt,dat[i],levels=clevs,cmap=cmp,extend='both')
    cs2=axs[i].contour(tme,zt,dat[i],levels=[0.5],linewidths=1,colors='k')
    axs[i].set_ylim(ylms[i])
    axs[i].set_yticks(yts[i])
    ts=axs[i].set_title(tts[i],fontsize=fs)
    ts.set_y(0.96)

axs[0].set_xlim(20000,0)
axs[0].set_xticks(np.arange(20000,-1,-2000))
axs[0].set_xticklabels(np.arange(20,-1,-2))

rght=0.85
plt.subplots_adjust(wspace=0.3,top=0.9,right=rght,hspace=0.2)
axs[2].set_xlabel('Time (ka BP)',fontsize=fs)
axs[2].xaxis.set_label_coords(1.15,-0.2)
axs[2].set_ylabel('Depth (m)',fontsize=fs)
axs[2].yaxis.set_label_coords(-0.3,1.1)
#ts=plt.suptitle(r'Ratio of Ideal Age to $\Delta^{14}$C Age')
#ts.set_x(0.48)
#ts.set_y(0.99)
#colorbar()
cax=fig.add_axes([rght+0.02, axs[-1].get_position().y0, 0.03, axs[0].get_position().y1-axs[-1].get_position().y0]) #x,y,w,h
cbar=plt.colorbar(cs,cax=cax,orientation='vertical')
cbar.set_label('Ideal Age / $\Delta^{14}$C Age',fontsize=fs,rotation=270,va='bottom')
cbar.ax.tick_params(labelsize=tick_size)

letts=['a)','b)','c)','d)']
for i in range(0,len(axs)):
    axs[i].annotate(letts[i],(0,1.03),xycoords='axes fraction',fontsize=fs)
    axs[i].tick_params(labelsize=tick_size)

plt.savefig(svdirec+'figure10.pdf')

### <b>Supplementary Figures

Get GLODAPv1 data and core data for this

In [ ]:
#GLODAP
dsg=xr.open_dataset(wrk_direc+'og_GLODAP/BkgC14_GLODAP1.nc')
lon=dsg.lon
lat=dsg.lat
levs=dsg.level
c14_glo=dsg.C14_BKG
#1˚ regridded model D14C
dsr=xr.open_dataset(wrk_direc+'citrace/ctrace_ABIO_D14Cocn_decadal_bilinear_regrid_1.0x1.0.nc')
tlatr=dsr.lat
tlonr=dsr.lon
ztr=dsr.z_t #already in m because I converted it when I regridded
D14C_r=dsr.ABIO_D14Cocn
#C14 age (for B-P)
D14C_r_age=cif.c14_age_godwin(D14C_r)

#Interpolate glodap to model depths, otherwise interp gets rightfully confused because
#glodap has fewer depths
glo_interp=c14_glo.interp(level=ztr)
#rename coords to be the same as model regrid
glo_interp=glo_interp.rename({'lat':'y','lon':'x'})
glo_interp=glo_interp.drop('level')
#Roll the glodap grid to start in the same place as the model grid
glo_interp=glo_interp.roll(x=180, roll_coords='True')

#CORE RECORDS
data_path=wrk_direc+'citrace/D14C_data/Zhaoetal2017data.txt'
data=pd.read_table(data_path,sep='\t')
#Pacific
pac_data=data[((data['lon.']>120) | (data['lon.']<-65)) & (data['cal.age']<22000)] 
#this must be the | operator to work, do not type 'or'

#Get all core records in your 4 domains:
#1. NWPAC: 135E-170W (190E), 30N-60N
#2. NEPAC: 150W (210E)-120W (240E), 30N-60N
#3. EEQPAC: 120W (240E)-80W (280E), 10S-10N
#4. SWPAC: 170E-170W (190E), 50S-30S

nwpac_data=pac_data[(pac_data['lon.']>135) | (pac_data['lon.']<-170)] 
#this must be the | or  & operator to work, do not type 'or', etc
nwpac=nwpac_data[(nwpac_data['lat.']>30) & (nwpac_data['lat.']<60.2)] #includes the Max et al 2014 core at 60.1N!

nepac_data=pac_data[(pac_data['lon.']>-150) & (pac_data['lon.']<-120)] 
#this must be the | or  & operator to work, do not type 'or', etc
nepac=nepac_data[(nepac_data['lat.']>30) & (nepac_data['lat.']<60)] 

eqpac_data=pac_data[(pac_data['lon.']>-120) & (pac_data['lon.']<-80)] 
#this must be the | or  & operator to work, do not type 'or', etc
epac=eqpac_data[(eqpac_data['lat.']>-10) & (eqpac_data['lat.']<10)] 

swpac_data=pac_data[(pac_data['lon.']>170) | (pac_data['lon.']<-170)] 
#this must be the | or  & operator to work, do not type 'or', etc
swpac=swpac_data[(swpac_data['lat.']>-50) & (swpac_data['lat.']<-30)] 

<b>Figure S1:</b> Time series of ice fraction in the Pacific sector of the Southern Ocean

In [ ]:
ifrac_pac=ds0.IFRAC.where(mask==2)
area=ds0.TAREA.where(mask==2)
ifrac_pac_ts=(ifrac_pac[:,0:13]*area[0:13]).sum(dim=('nlat','nlon'))/area[0:13].sum()

In [ ]:
tme=np.arange(22000,0,-10)

fig,ax=plt.subplots()
fig.set_size_inches(5,2)
fs=9
tick_size=7

for event in ['HS1','BA','YD']:
    ax.axvspan(xmin=paleo_events[event]['yrs'][0]*1000,xmax=paleo_events[event]['yrs'][1]*1000,
                    facecolor='None',edgecolor='gray',alpha=0.5)

ax.plot(tme,ifrac_pac_ts,color='k')
ax.set_title('Pacific Ice Fraction (south of 60$\degree$S)',fontsize=fs)

plt.subplots_adjust(bottom=0.2)

ax.set_xticks(np.arange(22000,-1,-2000))
ax.set_xticklabels(np.arange(22,-1,-2))
ax.set_xlim(20000,0)
ax.set_xlabel('Time (ka BP)',fontsize=fs)
ax.set_ylabel('Ice Fraction',fontsize=fs)
ax.set_yticks(np.arange(0.5,0.86,0.1))
ann=[ax.annotate(event,(paleo_events[event]['yrs'][0]*1000-0.5*1000*(paleo_events[event]['yrs'][0]-paleo_events[event]['yrs'][1]),0.83),
                        xycoords='data',fontsize=tick_size+1,ha='center',va='bottom',color='k',zorder=6) for event in ['HS1','BA','YD']]

plt.savefig(svdirec+'figureS1.pdf')

<b>Figure S2:</b> Model regridded zonal mean Pacific D14C vs GLODAP and difference

In [ ]:
#present (0ka)
D14C_0=D14C_r[-1,:,:,125:276].mean('x')
#GLODAP
D14C_glo=glo_interp[:,:,125:276].mean('x')
mo_diff=D14C_0-D14C_glo

In [ ]:
levs=np.arange(-250,-24,25)
cmap=plt.cm.inferno
tick_size=8
fs=9

fig,ax=plt.subplots(3,1,sharex=True)
fig.set_size_inches(3,4.2)
rght=0.7
plt.subplots_adjust(hspace=0.25,left=0.25,right=rght,top=0.95)
axs=ax.flatten()
cs0=axs[0].contourf(tlatr[:,0],-ztr,D14C_0,levels=levs,cmap=cmap,extend='both') #190.1E
ts0=axs[0].set_title('C-iTRACE',fontsize=fs)
ts0.set_y(0.95)
axs[0].set_facecolor('0.7')
axs[0].set_xlim(-80,60)
cs1=axs[1].contourf(tlatr[:,0],-ztr,D14C_glo,levels=levs,cmap=cmap) #depth,lat,lon 190.5E (169.5W)
ts1=axs[1].set_title('GLODAP',fontsize=fs)
ts1.set_y(0.95)
axs[1].set_xlim(-80,60)
axs[1].set_facecolor('0.7')
cax=fig.add_axes([rght+0.02, axs[1].get_position().y0, 0.03, axs[0].get_position().y1-axs[1].get_position().y0]) #x,y,w,h
cbar1=plt.colorbar(cs1,cax=cax,orientation='vertical')
cbar1.set_label(r'$\Delta^{14}$C (‰)',fontsize=fs,rotation=270,va='bottom')
cbar1.ax.tick_params(labelsize=tick_size)

cs3=axs[2].contourf(tlatr[:,0],-ztr,mo_diff,levels=np.arange(-75,76,10),extend='both',cmap=plt.cm.RdBu_r) #190.1E
cax2=fig.add_axes([rght+0.02, axs[-1].get_position().y0, 0.03, axs[-1].get_position().y1-axs[-1].get_position().y0]) #x,y,w,h
cbar2=plt.colorbar(cs3,cax=cax2,orientation='vertical')
cbar2.set_label('$\Delta^{14}$C (‰)\nDifference',fontsize=fs,rotation=270,va='bottom')
cbar2.ax.tick_params(labelsize=tick_size)
ts2=axs[2].set_title('C-iTRACE - GLODAP',fontsize=fs)
ts2.set_y(0.95)
axs[2].set_facecolor('0.7')
axs[2].set_xlim(-80,60)
axs[1].set_ylabel('Depth (m)',fontsize=fs)
#axs[-1].set_xlabel('Latitude',fontsize=fs-1)
axs[-1].set_xticklabels(['','50$\degree$S','0$\degree$','50$\degree$N'])
letts=['a)','b)','c)']
for i in range(0,3):
    axs[i].tick_params(labelsize=tick_size)
    axs[i].set_yticks([-5000,-2500,0])
    axs[i].annotate(letts[i],(-0.25,1.045),xycoords='axes fraction',fontsize=fs)

#ts=plt.suptitle(r'Pacific zonal mean $\Delta^{14}$C')
#ts.set_x(0.5)
plt.savefig(svdirec+'figureS2.pdf')

<b>Figure S3:</b> Four regions: Model regridded Pacific D14C age (raw and modern GLODAP corrected) vs cores over time

In [ ]:
#Get 'corrected' model D14C age
model_bias=D14C_r[-1]-glo_interp
model_bias=model_bias.drop(['lat','lon','z_t'])
model_corrected=D14C_r-model_bias
mc_age=cif.c14_age(model_corrected)

#Get the regions in the model
nwp=D14C_age[:,:,regions['NWP']['yid'][0]:regions['NWP']['yid'][1],regions['NWP']['xid'][0]:regions['NWP']['xid'][1]]
nep=D14C_age[:,:,regions['NEP']['yid'][0]:regions['NEP']['yid'][1],regions['NEP']['xid'][0]:regions['NEP']['xid'][1]]
eqp=D14C_age[:,:,regions['EEP']['yid'][0]:regions['EEP']['yid'][1],regions['EEP']['xid'][0]:regions['EEP']['xid'][1]]
swp=D14C_age[:,:,regions['SWP']['yid'][0]:regions['SWP']['yid'][1],regions['SWP']['xid'][0]:regions['SWP']['xid'][1]]
#Calculate the region mean D14C age
nwpm=nwp.mean(dim=('nlat','nlon'))
nepm=nep.mean(dim=('nlat','nlon'))
eqpacm=eqp.mean(dim=('nlat','nlon'))
swpm=swp.mean(dim=('nlat','nlon'))

a967=swpac[(swpac['ref.']=='Shao et al. (2019)') & (swpac['water.depth']==967.0)]
a967=a967[a967['benthic.14C_age']-a967['14C age']>0] #yes
a967=a967.sort_values(by='cal.age',ascending=False)
a1364=epac[(epac['ref.']=='Stott et al. (2019)') & (epac['water.depth']==1364.0)] #yes
a1364=a1364.sort_values(by='cal.age',ascending=False)
a682=nepac[((nepac['ref.']=='Walczak et al. (2020)')|(nepac['ref.']=='Davies-Walczak et al. (2014)')) & (nepac['water.depth']==682.0)] #yes
a682=a682.sort_values(by='cal.age',ascending=False)
#a1215=nwpac[((nwpac['ref.']=='Okazaki (2014)')|(nwpac['ref.']=='Ikehara et al. (2006)')) & ((nwpac['water.depth']>1210.0)& (nwpac['water.depth']<=1215.0))] #yes
#a1366=nwpac[(nwpac['ref.']=='Ahagon et al. (2003)') & (nwpac['water.depth']==1366.0)] #yes
a2215=nwpac[(nwpac['ref.']=='Minoshima et al. (2007)') & (nwpac['water.depth']==2215.0)] #yes
a2101=nwpac[(nwpac['ref.']=='Okazaki et al. (2012)') & (nwpac['water.depth']==2101.0)] #yes when combined with Minoshima et al 2007
a2150=pd.concat([a2101,a2215])
a2150=a2150.sort_values(by='cal.age',ascending=False)

In [ ]:
#Core locations and averaging regions
#Get single points for cores (lat/lon is repeated for each depth; no need to plot the repeats)
nw=a2150.groupby(['lon.', 'lat.']).size().rename('count').reset_index()
ne=a682.groupby(['lon.', 'lat.']).size().rename('count').reset_index()
eq=a1364.groupby(['lon.', 'lat.']).size().rename('count').reset_index()
sw=a967.groupby(['lon.', 'lat.']).size().rename('count').reset_index()
core=[nw,ne,eq,sw]

#Get single points for cores (lat/lon is repeated for each depth; no need to plot the repeats)
nw_all=nwpac.groupby(['lon.', 'lat.']).size().rename('count').reset_index()
ne_all=nepac.groupby(['lon.', 'lat.']).size().rename('count').reset_index()
eq_all=epac.groupby(['lon.', 'lat.']).size().rename('count').reset_index()
sw_all=swpac.groupby(['lon.', 'lat.']).size().rename('count').reset_index()

tick_size=9
fs=10
fig=plt.figure(constrained_layout=False)
gs0=fig.add_gridspec(nrows=1,ncols=1,bottom=0.7,left=0.11,top=0.97,right=0.97)
gs1=fig.add_gridspec(nrows=2,ncols=4,hspace=0.2,wspace=0.5,top=0.61,left=0.11,right=0.97)
fig.set_size_inches(6.3,5.5)
#map axes
ax=fig.add_subplot(gs0[:],projection=ccrs.PlateCarree(central_longitude=200))
ax.set_extent([100,310,-90, 90], ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, facecolor='0.8',edgecolor='None',linewidth=0.5)
#time series axes
axs=[fig.add_subplot(gs1[0,i:i+2]) for i in [0,2]]+[fig.add_subplot(gs1[1,i:i+2]) for i in [0,2]]

#all cores
b1_cores=ax.scatter(nw_all['lon.'],nw_all['lat.'], fc='cornflowerblue',alpha=0.5,transform=ccrs.PlateCarree(),s=3)
b2_cores=ax.scatter(ne_all['lon.'],ne_all['lat.'],fc='cornflowerblue',alpha=0.5,transform=ccrs.PlateCarree(),s=3)
b3_cores=ax.scatter(eq_all['lon.'],eq_all['lat.'], fc='cornflowerblue',alpha=0.5,transform=ccrs.PlateCarree(),s=3)
b4_cores=ax.scatter(sw_all['lon.'],sw_all['lat.'], fc='cornflowerblue',alpha=0.5,transform=ccrs.PlateCarree(),s=3)
for i in range(0,4):
    ax.scatter(core[i]['lon.'],core[i]['lat.'], marker='*', fc='darkmagenta',transform=ccrs.PlateCarree(),s=18)
#get the second core location for the NW Pac
ax.scatter(core[0]['lon.'],core[0]['lat.'], marker='*', fc='darkmagenta',transform=ccrs.PlateCarree(),s=18)
t0=ax.set_title('Core Locations',fontsize=fs)
t0.set_y(0.97)
#averaging boxes
b1=ax.plot([135,190,190,135,135],[30,30,63,63,30],color='k',linewidth=1,transform=ccrs.PlateCarree()) #lower left start, clockwise
b2=ax.plot([209,241,241,209,209],[30,30,63,63,30],color='k',linewidth=1,transform=ccrs.PlateCarree()) #lower left start, clockwise
b3=ax.plot([240,281,281,240,240],[-10,-10,12,12,-10],color='k',linewidth=1,transform=ccrs.PlateCarree()) #lower left start, clockwise
b4=ax.plot([170,191,191,170,170],[-50,-50,-30,-30,-50],color='k',linewidth=1,transform=ccrs.PlateCarree()) #lower left start, clockwise
gl=ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='gray', alpha=0.5, linestyle='-')
gl.xlabels_top=False #don't label the top or right with lat/lon
gl.ylabels_right=False
gl.xlines=False #turn off the grid lines in x
gl.ylines=False #turn off the grid lines in y
gl.xlocator=mticker.FixedLocator([120,180,-120,-60]) #these need not be evenly spaced
gl.ylocator=mticker.FixedLocator([-90,-60,-30,0,30,60,90])
gl.xformatter=LONGITUDE_FORMATTER
gl.yformatter=LATITUDE_FORMATTER
gl.xlabel_style = {'size': tick_size, 'color': 'k'}
gl.ylabel_style = {'size': tick_size, 'color': 'k'}

#model and corrected D14C age
i_index=[144,215,275,177]
j_index=[129,148,97,45]
k_index=[46,36,42,39]
age=[D14C_r_age[:,k,j,i]-D14C_r_age[:,0,j,i] for i,j,k in zip(i_index,j_index,k_index)]
age_corrected=[mc_age[:,k,j,i]-mc_age[:,0,j,i] for i,j,k in zip(i_index,j_index,k_index)]

#Cores
core=[a2150,a682,a1364,a967]
c1=a2150['site.name'].unique()
c2=a682['site.name'].unique()
c3=a1364['site.name'].unique()
c4=a967['site.name'].unique()
core_name=[c1[0]+' &\n '+c1[1],c2[0]+' &\n '+c2[1],c3[0],c4[0]+' & '+c4[1]]

tme=np.arange(22000,0,-10)
ymax=[2800,2000,2500,1150]
#depth=[-zt[46],-zt[36],-zt[42],-zt[39]] #model depths
depth=[2158,682,1364,967] #core depths
tts=['NW Pacific ', 'NE Pacific ', 
     'East Pacific ', 'SW Pacific ']
lbs1=['','','','Adjusted']
lbs2=['','','','Model']
locs=['lower right','upper right','upper right','upper right']

for i in range(0,4):
    plt.sca(axs[i])
    for event in ['HS1','BA','YD']:
        axs[i].axvspan(xmin=paleo_events[event]['yrs'][0]*1000,xmax=paleo_events[event]['yrs'][1]*1000,
                      facecolor='cornflowerblue',edgecolor='darkgoldenrod',alpha=0.25)
    #plot corrected model age in gray    
    axs[i].plot(tme,age_corrected[i],color='0.5',label=lbs1[i],lw=1)
    #plot the uncorrected model age in black
    axs[i].plot(tme,age[i],color='k',label=lbs2[i],lw=1)
    #plot the core
    axs[i].plot(core[i]['cal.age'],core[i]['benthic.14C_age']-core[i]['14C age'],color='0.7',
         lw=0.75, marker='o', mfc='darkmagenta', mec='None',label=core_name[i],markersize=4.5)
    err=[max(a,b) for a,b in zip(core[i]['benthic.14C_age_err'],core[i]['14C age_err '])]
    axs[i].errorbar(core[i]['cal.age'],core[i]['benthic.14C_age']-core[i]['14C age'],yerr=err, linestyle="None",color='darkmagenta',lw=1)
    axs[i].set_ylim([0,ymax[i]])
    axs[i].legend(loc=locs[i],frameon=False,handlelength=1,
              handletextpad=0.3,borderaxespad=0.01,labelspacing=0.1,
              fontsize=8) 
    ts=axs[i].set_title(tts[i]+'('+str('%i' %depth[i])+' m)',fontsize=fs)
    ts.set_y(0.97)
    
    axs[i].set_xticks(np.arange(22000,-1,-2000))
    axs[i].set_xticklabels(np.arange(22,-1,-2))
    axs[i].set_xlim(20000,0)
    if i<2:
        axs[i].set_xticklabels('')
    axs[i].tick_params(labelsize=tick_size)
axs[2].set_xlabel('Time (ka BP)')
axs[2].set_ylabel('B$-$P $\Delta^{14}$C Age (yr)')
axs[2].xaxis.set_label_coords(1.1,-0.25)
axs[2].yaxis.set_label_coords(-0.2,1.15)

ann=[axs[1].annotate(event,(paleo_events[event]['yrs'][0]*1000-0.5*1000*(paleo_events[event]['yrs'][0]-paleo_events[event]['yrs'][1]),1800),
                        xycoords='data',fontsize=6.5,ha='center',va='bottom',color='k',zorder=6) for event in ['HS1','BA','YD']]

letts=['a)','b)','c)','d)','e)']
ax.annotate(letts[0],(0,1.035),xycoords='axes fraction',fontsize=fs)
for i in range(0,len(axs)):
    axs[i].annotate(letts[i+1],(0,1.045),xycoords='axes fraction',fontsize=fs)
plt.savefig(svdirec+'figureS3.pdf')

<b> Figure S4:</b> D14C age (B-P) in the four regions compared to B-P from all cores in the region

In [ ]:
#NWPAC
#Step 1 get all the individual references
nwrefs=nwpac.groupby(['ref.']).size().rename('count').reset_index()
#Loop over references to get individual core time points;
#Don't actually need to do this because pandas can handle it
#but it's legacy from the detrending code and I don't care to
#rewrite it
nw_ages=[]
nw_times=[]
nw_depths=[]
for ref in nwrefs['ref.']:
    a=nwpac[nwpac['ref.']==ref]
    #Sort everything so calendar age is descending to PD
    a=a.sort_values(by='cal.age',ascending=False)
    #compute benthic minus planktonic ages
    a_bp=a['benthic.14C_age']-a['14C age']
    nw_ages.append(a_bp)
    nw_times.append(a['cal.age'])
    nw_depths.append(a['water.depth'])
nw_ages=np.concatenate(nw_ages)
nw_times=np.concatenate(nw_times)
nw_depths=np.concatenate(nw_depths)

#NEPAC
#Step 1 get all the individual references
nerefs=nepac.groupby(['ref.']).size().rename('count').reset_index()
#Loop over references to get individual core time points
ne_ages=[]
ne_times=[]
ne_depths=[]
for ref in nerefs['ref.']:
    a=nepac[nepac['ref.']==ref]
    #Sort everything so calendar age is descending to PD
    a=a.sort_values(by='cal.age',ascending=False)
    #compute benthic minus planktonic ages
    a_bp=a['benthic.14C_age']-a['14C age']
    #get rid of nans
    a_bp=a_bp.values
    a_bp2=a_bp[~np.isnan(a_bp)]
    tms=a['cal.age'].values
    tms=tms[~np.isnan(a_bp)]
    dpths=a['water.depth'].values
    dpths=dpths[~np.isnan(a_bp)]
    ne_ages.append(a_bp2)
    ne_times.append(tms)
    ne_depths.append(dpths)
ne_ages=np.concatenate(ne_ages)
ne_times=np.concatenate(ne_times)
ne_depths=np.concatenate(ne_depths)

#EEPAC
#Step 1 get all the individual references
eerefs=epac.groupby(['ref.']).size().rename('count').reset_index()
#Loop over references to get individual core time points
ee_ages=[]
ee_times=[]
ee_depths=[]
for ref in eerefs['ref.']:
    a=epac[epac['ref.']==ref]
    #Sort everything so calendar age is descending to PD
    a=a.sort_values(by='cal.age',ascending=False)
    #compute benthic minus planktonic ages
    a_bp=a['benthic.14C_age']-a['14C age']
    #get rid of nans
    a_bp=a_bp.values
    a_bp2=a_bp[~np.isnan(a_bp)]
    tms=a['cal.age'].values
    tms=tms[~np.isnan(a_bp)]
    dpths=a['water.depth'].values
    dpths=dpths[~np.isnan(a_bp)]
    ee_ages.append(a_bp2)
    ee_times.append(tms)
    ee_depths.append(dpths)
ee_ages=np.concatenate(ee_ages)
ee_times=np.concatenate(ee_times)
ee_depths=np.concatenate(ee_depths)

#SWPAC
#Step 1 get all the individual references
swrefs=swpac.groupby(['ref.']).size().rename('count').reset_index()
#Loop over references to get individual core time points
sw_ages=[]
sw_times=[]
sw_depths=[]
for ref in swrefs['ref.']:
    a=swpac[swpac['ref.']==ref]
    #Sort everything so calendar age is descending to PD
    a=a.sort_values(by='cal.age',ascending=False)
    #compute benthic minus planktonic ages
    a_bp=a['benthic.14C_age']-a['14C age']
    #get rid of nans
    a_bp=a_bp.values
    a_bp2=a_bp[~np.isnan(a_bp)]
    tms=a['cal.age'].values
    tms=tms[~np.isnan(a_bp)]
    dpths=a['water.depth'].values
    dpths=dpths[~np.isnan(a_bp)]
    sw_ages.append(a_bp2)
    sw_times.append(tms)
    sw_depths.append(dpths)
sw_ages=np.concatenate(sw_ages)
sw_times=np.concatenate(sw_times)
sw_depths=np.concatenate(sw_depths)

In [ ]:
#Get 'corrected' model D14C age
model_bias=D14C_r[-1]-glo_interp
model_bias=model_bias.drop(['lat','lon','z_t'])
model_corrected=D14C_r-model_bias
mc_age=cif.c14_age(model_corrected)

#Get the regions in the model
nwpc=mc_age[:,:,regions['NWP_r']['yid'][0]:regions['NWP_r']['yid'][1],regions['NWP_r']['xid'][0]:regions['NWP_r']['xid'][1]]
nepc=mc_age[:,:,regions['NEP_r']['yid'][0]:regions['NEP_r']['yid'][1],regions['NEP_r']['xid'][0]:regions['NEP_r']['xid'][1]]
eqpc=mc_age[:,:,regions['EEP_r']['yid'][0]:regions['EEP_r']['yid'][1],regions['EEP_r']['xid'][0]:regions['EEP_r']['xid'][1]]
swpc=mc_age[:,:,regions['SWP_r']['yid'][0]:regions['SWP_r']['yid'][1],regions['SWP_r']['xid'][0]:regions['SWP_r']['xid'][1]]
#Calculate the region mean D14C age
nwpm_c=nwpc.mean(dim=('y','x'))
nepm_c=nepc.mean(dim=('y','x'))
eqpm_c=eqpc.mean(dim=('y','x'))
swpm_c=swpc.mean(dim=('y','x'))

In [ ]:
dat=[(nwpm_c-nwpm_c[:,0]).T,(nepm_c-nepm_c[:,0]).T,(eqpm_c-eqpm_c[:,0]).T,(swpm_c-swpm_c[:,0]).T]
ages=[nw_ages,ne_ages,ee_ages,sw_ages]
tme=np.arange(22000,0,-20) #subsample here because this figure is huge otherwise
zt=-0.01*ds0['z_t']
depths=[zt,zt,zt,zt]
depths2=[-nw_depths,-ne_depths,-ee_depths,-sw_depths]
tmes=[nw_times,ne_times,ee_times,sw_times]
#cmp=plt.cm.bone_r
cmp=plt.cm.RdBu_r
#model has deeper depths but cores don't go as deep in NPac
yts=[np.arange(-4000,1,1000),np.arange(-4000,1,1000),np.arange(-4000,1,1000),np.arange(-5000,1,1000)]
ylms=[[-4200,0],[-4200,0],[-4125,0],[-5125,0]]
tts=['NW Pacific','NE Pacific','East Pacific','SW Pacific']
fs=9
tick_size=7

fig,ax=plt.subplots(2,2,sharex=True)
fig.set_size_inches(6,3.75)
axs=ax.flatten()
for i in range(0,4):
    cs=axs[i].pcolormesh(tme,depths[i],dat[i][:,::2],cmap=plt.cm.viridis,vmin=0,vmax=2500)
    sc1=axs[i].scatter(tmes[i],depths2[i],c=ages[i],edgecolors='k',s=5,linewidths=0.5,
                       cmap=plt.cm.viridis,vmin=0,vmax=2500)
    axs[i].set_ylim(ylms[i])
    axs[i].set_yticks(yts[i])
    ts=axs[i].set_title(tts[i],fontsize=fs)
    ts.set_y(0.96)

axs[0].set_xticks(np.arange(22000,-1,-2000))
axs[0].set_xticklabels(np.arange(22,-1,-2))
axs[0].set_xlim(20000,0)

rght=0.82
plt.subplots_adjust(wspace=0.3,top=0.9,right=rght,hspace=0.2)
axs[2].set_xlabel('Time (ka BP)',fontsize=fs)
axs[2].xaxis.set_label_coords(1.15,-0.2)
axs[2].set_ylabel('Depth (m)',fontsize=fs)
axs[2].yaxis.set_label_coords(-0.3,1.1)
#ts=plt.suptitle(r'Ratio of Ideal Age to $\Delta^{14}$C Age')
#ts.set_x(0.48)
#ts.set_y(0.99)
#colorbar()
cax=fig.add_axes([rght+0.02, axs[-1].get_position().y0, 0.03, axs[0].get_position().y1-axs[-1].get_position().y0]) #x,y,w,h
cbar=plt.colorbar(cs,cax=cax,orientation='vertical')
cbar.set_label('B-P $\Delta^{14}$C Age',fontsize=fs,rotation=270,va='bottom')
cbar.ax.tick_params(labelsize=tick_size)

letts=['a)','b)','c)','d)']
for i in range(0,len(axs)):
    axs[i].annotate(letts[i],(0,1.03),xycoords='axes fraction',fontsize=fs)
    axs[i].tick_params(labelsize=tick_size)

plt.savefig(svdirec+'figureS4.pdf')

<b>Figure S5:</b> Four regions: Model Pacific D14C standard deviation vs mean over time (hovmöllers). Don't forget to read in the model D14C to make this figure!

In [ ]:
#Get the regions in the model
nwp=D14C[:,:,regions['NWP']['yid'][0]:regions['NWP']['yid'][1],regions['NWP']['xid'][0]:regions['NWP']['xid'][1]]
nep=D14C[:,:,regions['NEP']['yid'][0]:regions['NEP']['yid'][1],regions['NEP']['xid'][0]:regions['NEP']['xid'][1]]
eqpac=D14C[:,:,regions['EEP']['yid'][0]:regions['EEP']['yid'][1],regions['EEP']['xid'][0]:regions['EEP']['xid'][1]]
swp=D14C[:,:,regions['SWP']['yid'][0]:regions['SWP']['yid'][1],regions['SWP']['xid'][0]:regions['SWP']['xid'][1]]
#Calculate the region mean D14C
nwpm=nwp.mean(dim=('nlat','nlon'))
nepm=nep.mean(dim=('nlat','nlon'))
eqpacm=eqpac.mean(dim=('nlat','nlon'))
swpm=swp.mean(dim=('nlat','nlon'))
#Calculate the spatial std dev
nwpst=nwp.std(dim=('nlat','nlon'))
nepst=nep.std(dim=('nlat','nlon'))
epacst=eqpac.std(dim=('nlat','nlon'))
swpst=swp.std(dim=('nlat','nlon'))

In [ ]:
tme=np.arange(22000,0,-10)
zt=-0.01*ds0['z_t']
cmp=plt.cm.magma
clevs=np.arange(0,51,5)

fig,ax=plt.subplots(2,2,sharex=True)
fig.set_size_inches(8,5)
axs=ax.flatten()

cs1=axs[0].contourf(tme,zt,nwpst.T,levels=clevs,extend='max',cmap=cmp)
axs[0].set_ylim(-5375,0)
axs[0].set_yticks(np.arange(-5000,1,1000))
axs[0].set_title('NW Pacific',fontsize=10)

cs2=axs[1].contourf(tme,zt,nepst.T,levels=clevs,extend='max',cmap=cmp)
axs[1].set_ylim(-5375,0)
axs[1].set_yticks(np.arange(-5000,1,1000))
axs[1].set_title('NE Pacific',fontsize=10)

cs3=axs[2].contourf(tme,zt,epacst.T,levels=clevs,extend='max',cmap=cmp)
axs[2].set_ylim(-4125,0)
axs[2].set_yticks(np.arange(-4000,1,1000))
axs[2].set_title('East Pacific',fontsize=10)

cs4=axs[3].contourf(tme,zt,swpst.T,levels=clevs,extend='max',cmap=cmp)
axs[3].set_ylim(-5125,0)
axs[3].set_yticks(np.arange(-5000,1,1000))
axs[3].set_title('SW Pacific',fontsize=10)

axs[0].set_xticks(np.arange(22000,-1,-2000))
axs[0].set_xticklabels(np.arange(22,-1,-2))
axs[0].set_xlim(20000,0)

rght=0.85
plt.subplots_adjust(wspace=0.3,top=0.9,right=rght)
axs[2].set_xlabel('Time (ka BP)',fontsize=10)
axs[2].xaxis.set_label_coords(1.15,-0.2)
axs[2].set_ylabel('Depth (m)',fontsize=10)
axs[2].yaxis.set_label_coords(-0.3,1.1)
#ts=plt.suptitle(r'Model $\Delta^{14}$C spatial std. dev.')
#ts.set_x(0.48)
#ts.set_y(0.99)
#colorbar()
cax=fig.add_axes([rght+0.02, axs[-1].get_position().y0, 0.03, axs[0].get_position().y1-axs[-1].get_position().y0]) #x,y,w,h
cbar=plt.colorbar(cs3,cax=cax,orientation='vertical')
cbar.set_label('$\sigma_{spatial}$ $\Delta^{14}$C (‰)',fontsize=10,rotation=270,va='bottom')
cbar.ax.tick_params(labelsize=8)
letts=['a)','b)','c)','d)']
for i in range(0,len(axs)):
    axs[i].annotate(letts[i],(0,1.05),xycoords='axes fraction',fontsize=fs)
    axs[i].tick_params(labelsize=8)
plt.savefig(svdirec+'figureS5.pdf')

### <b> Not shown figures and calculations used to support statements in the text</b>

<b> Figure NS1:</b> No evidence of dense water formation in Bering Sea or Sea of Okhotsk

In [ ]:
#Bering Sea
#Get the regions in the model
rho=1000*ds0.PD-1000 #in kg/m3
ber=rho[:,:,77:83,54:65].mean(dim=('nlat','nlon')) 
bermld_max=0.01*ds0.XMXL[:,77:83,54:65].mean(dim=('nlat','nlon'))
bermld=0.01*ds0.HMXL[:,77:83,54:65].mean(dim=('nlat','nlon'))

In [ ]:
#Sea Okhotsk
#Get the regions in the model
rho=1000*ds0.PD-1000 #in kg/m3
okh=rho[:,:,76:83,49:55].mean(dim=('nlat','nlon')) 
okhmld_max=0.01*ds0.XMXL[:,76:83,49:55].mean(dim=('nlat','nlon'))
okhmld=0.01*ds0.HMXL[:,76:83,49:55].mean(dim=('nlat','nlon'))

In [ ]:
tme=np.arange(22000,0,-10)
zt=-0.01*ds0['z_t']
levs=np.arange(27.4,29.61,0.2)

fig,ax=plt.subplots(2,1,sharex=True)
fig.set_size_inches(6,4)
axs=ax.flatten()

for event in ['HS1','BA','YD']:
    axs[0].axvspan(xmin=paleo_events[event]['yrs'][0]*1000,xmax=paleo_events[event]['yrs'][1]*1000,
            facecolor='0.7',edgecolor='k',alpha=0.5)
        
axs[0].plot(tme,okhmld_max,color='indigo',lw=1,label='Max MLD')
axs[0].plot(tme,okhmld,color='g',lw=1,label='MLD')
axs[0].set_title('Sea of Okhotsk MLD (m)')
axs[0].legend(loc='upper right',frameon=False,fontsize=10,borderaxespad=0)
cs=axs[1].contourf(tme,zt,okh.T,levels=levs,extend='both',cmap=plt.cm.viridis_r)
#ax2=axs[1].twinx()
for event in ['HS1','BA','YD']:
    axs[1].axvspan(xmin=paleo_events[event]['yrs'][0]*1000,xmax=paleo_events[event]['yrs'][1]*1000,
            facecolor='None',edgecolor='k',alpha=1)
#ax2=axs[0].twinx()    
#ax2.plot(tme,-bermld_max,color='k',lw=1)
#ax2.plot(tme,bermld_max,color='g',lw=0.7)
#ax2.set_ylim(0,50)
axs[0].set_xlim(22000,0)
#axs[0].set_ylim(0,13)
axs[1].set_title('Sea of Okhotsk Density')
axs[1].set_ylim(-4800,0)
axs[1].set_yticks([-4500,-3000,-1500,0])

ann=[axs[0].annotate(event,(paleo_events[event]['yrs'][0]*1000-0.5*1000*(paleo_events[event]['yrs'][0]-paleo_events[event]['yrs'][1]),11.5),
            xycoords='data',fontsize=9,ha='center',va='bottom',color='k',zorder=6) for event in ['HS1','BA','YD']]

bot=0.23
plt.subplots_adjust(bottom=bot,top=0.94)
cax=fig.add_axes([axs[-1].get_position().x0, axs[-1].get_position().y0-0.1,axs[-1].get_position().x1-axs[-1].get_position().x0,0.03]) #x,y,w,h
cbar=plt.colorbar(cs,cax=cax,orientation='horizontal')
cbar.set_label(r'$\sigma_{0}$ (kg m$^{-3}$)',fontsize=11,va='top')

#plt.savefig(svdirec+'okhotsk_rho_and_max_mld.pdf')

<b> Computation 1: </b> B-A, B-P, Ideal, and Ventilation age differences at beginning and end of HS1 in EEP (Eastern Equatorial Pacific)

In [ ]:
#D14C B-A and for B-P, E Eq Pac
region='EEP'
nwp=D14C_ba_age[:,:,regions[region]['yid'][0]:regions[region]['yid'][1],regions[region]['xid'][0]:regions[region]['xid'][1]]
nwp1=D14C_age[:,:,regions[region]['yid'][0]:regions[region]['yid'][1],regions[region]['xid'][0]:regions[region]['xid'][1]]
#Ideal
nwpi=iage[:,:,regions[region]['yid'][0]:regions[region]['yid'][1],regions[region]['xid'][0]:regions[region]['xid'][1]]
#Vent
nwpv=vage[:,:,regions[region]['yid'][0]:regions[region]['yid'][1],regions[region]['xid'][0]:regions[region]['xid'][1]]


#Calculate the region mean D14C age (B-A)
nwpm=nwp.mean(dim=('nlat','nlon'))
#Calculate the region mean D14C age (no atmo correction for B-P)
nwpm1=nwp1.mean(dim=('nlat','nlon'))
#Calculate the region mean ideal age
nwpmi=nwpi.mean(dim=('nlat','nlon'))
#Calculate the region mean ventilation age
nwpmv=nwpv.mean(dim=('nlat','nlon'))

In [ ]:
#Get HS1 indices
i=54 #depth index, -1 or other (54,58) if not in N Pac
#Indices are for computations as well!
hs1=paleo_events['HS1']['idx']
#B-A D14C age difference
diffc=nwpm[hs1[-1],i]-nwpm[hs1[0],i] #bottom
#ideal age difference
diffi=nwpmi[hs1[-1],i]-nwpmi[hs1[0],i] #bottom
#ventilation age difference
diffv=nwpmv[hs1[-1],i]-nwpmv[hs1[0],i] #bottom
#B-P D14C age difference
diffc1=(nwpm1[hs1[-1],i]-nwpm1[hs1[0],i])-(nwpm1[hs1[-1],0]-nwpm1[hs1[0],0])

In [ ]:
#Print the age differences
diffs=[diffc,diffi,diffv,diffc1]
for d in diffs:
    print(d)